# CyprusDB generation

This notebook contains all the steps followed to generate the CyprusDB dataset. This ensures its reproducibility.

## Data sources

1) [CyStat's 2011 census](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)
2) [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

# Preparation

In [1]:
### Imports ###

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import folium

# Data retrieval
import requests
import json

# Data reading
import tabula

# Others
import time
from pprint import pprint
import os
import glob
from datetime import date

# 1) Population list

Source: [CyStat](https://www.data.gov.cy/dataset/%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CF%8C%CF%82-%CE%BA%CE%B1%CF%84%CE%AC-%CF%84%CF%8C%CF%80%CE%BF-%CE%B4%CE%B9%CE%B1%CE%BC%CE%BF%CE%BD%CE%AE%CF%82-%CE%B1%CF%80%CE%BF%CE%B3%CF%81%CE%B1%CF%86%CE%AE-%CF%80%CE%BB%CE%B7%CE%B8%CF%85%CF%83%CE%BC%CE%BF%CF%8D-2011)

The base town list is extracted from the above link. The list is then cleaned and the population is extracted from the excel file. The complete list of settlements used in this project is taken from the sheet `Γ2`.

In [2]:
# Open raw file
filepath = r'sources/Census 2011 Excel format/POP_CEN_11-POP_PLACE_RESID-EL-171115.xls'

# Read file
raw_census_df = pd.read_excel(filepath, sheet_name='Γ2', skiprows=3)
raw_census_df.head()

,Unnamed: 0,ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ,"ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ",ΝΟΙΚΟΚΥΡΙΑ,Unnamed: 4,Unnamed: 5,Unnamed: 6,ΙΔΡΥΜΑΤΑ,Unnamed: 8,Unnamed: 9,Unnamed: 10,ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,ΑΡΙΘΜΟΣ,ΠΛΗΘΥΣΜΟΣ,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Σύνολο,Άνδρες,Γυναίκες,NaN,Σύνολο,Άνδρες,Γυναίκες,Σύνολο,Άνδρες,Γυναίκες
2,NaN,NaN,Σύνολο,303242,836566,407228,429338,211,3841,1552,2289,840407,408780,431627
3,NaN,1,Επαρχία Λευκωσίας,119203,324952,157307,167645,94,2028,955,1073,326980,158262,168718
4,NaN,1000,Δήμος Λευκωσίας,22833,54452,26086,28366,11,562,434,128,55014,26520,28494


## Data cleaning

In [3]:
# Retain only relevant columns
to_retain = [
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ',
    'Unnamed: 12', # Males
    'Unnamed: 13' # Fermales
]

census_df = raw_census_df[to_retain].copy()

# Rename columns
to_rename = {
    'ΓΕΩΓ/ΚΟΣ ΚΩΔΙΚΟΣ': 'geo_code',
    'ΕΠΑΡΧΙΑ, ΔΗΜΟΣ/ΚΟΙΝΟΤΗΤΑ ΚΑΙ ΕΝΟΡΙΑ': 'town',
    'ΣΥΝΟΛΟ ΠΛΗΘΥΣΜΟΥ': 'population',
    'Unnamed: 12' : 'male_population',
    'Unnamed: 13' : 'female_population'
}

census_df.rename(columns=to_rename, inplace=True)

# Drop the three first rows for format purposes
census_df.drop([0, 1, 2], inplace=True)

# Drop the last six rows, which provide no information
census_df.drop(census_df.tail(6).index, inplace=True)

# Fill NaNs with 0 in male and female population columns
census_df.fillna(value = {'male_population' : 0, 'female_population' : 0}, inplace=True)

census_df.head()

,geo_code,town,population,male_population,female_population
3,1,Επαρχία Λευκωσίας,326980,158262,168718
4,1000,Δήμος Λευκωσίας,55014,26520,28494
5,100001,Άγιος Ανδρέας,5767,2817,2950
6,100002,Τρυπιώτης,2158,983,1175
7,100003,Νεμπέτ Χανέ,189,86,103


### Create the 'district' column

In [4]:
# Extract the district from the 'town' column and create a new column
# if 'Επαρχία' is in the town name, then the district is the town name
census_df['district'] = census_df['town'].apply(lambda x: x.split(' ')[1] if 'Επαρχία' in x else np.nan)

# Fill the NaN values with the previous value
# using a forward fill
census_df['district'].fillna(method='ffill', inplace=True)

# Remove the rows that contain the district name
census_df = census_df[~census_df['town'].str.contains('Επαρχία')]

census_df.head()

,geo_code,town,population,male_population,female_population,district
4,1000,Δήμος Λευκωσίας,55014,26520,28494,Λευκωσίας
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας


### Include suburbs into the main cities
This requieres reversing the genitives of the dimos to the nominative case. Since the number of suburbs is small, we can do this manually


In [5]:
# Map genitive town names to nominative town names
dimos_manual_mapping = {
    # Nicosia
    'Δήμος Λευκωσίας' : 'Λευκωσία',
    'Δήμος Αγίου Δομετίου' : 'Άγιος Δομέτιος',
    'Δήμος Έγκωμης' : 'Έγκωμη',
    'Δήμος Στροβόλου' : 'Στροβόλος',
    'Δήμος Αγλαντζιάς' : 'Αγλαντζιά',
    'Δήμος Λακατάμειας' : 'Λακατάμεια',
    'Δήμος Λατσιών' : 'Λατσιά',
    'Δήμος Ιδαλίου' : 'Δάλι',
    # Larnaka
    'Δήμος Λάρνακας' : 'Λάρνακα',
    'Δήμος Αραδίππου' : 'Αραδίππου',
    # Limassol
    'Δήμος Λεμεσού' : 'Λεμεσός',
    'Δήμος Μέσα Γειτονιάς' : 'Μέσα Γειτονιά',
    'Δήμος Αγίου Αθανασίου' : 'Άγιος Αθανάσιος',
    'Δήμος Γερμασόγειας' : 'Γερμασόγεια',
    'Δήμος Κάτω Πολεμιδιών' : 'Κάτω Πολεμίδια',
    # Paphos
    'Δήμος Πάφου' : 'Πάφος',
    'Δήμος Γεροσκήπου' : 'Γεροσκήπου',
    'Δήμος Πόλεως Χρυσοχούς' : 'Πόλις Χρυσοχούς',
    'Δήμος Πέγειας' : 'Πέγεια'
}

# Create a column indicating whether the town name is a dimos
census_df['is_dimos'] = census_df['town'].apply(lambda x: True if x in dimos_manual_mapping.keys() else False)

# Replace dimos names with settlement names
census_df['town'] = census_df['town'].apply(lambda x: dimos_manual_mapping[x] if x in dimos_manual_mapping.keys() else x)

In [10]:
census_df.head(5)

,geo_code,town,population,male_population,female_population,district,is_dimos
4,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
5,100001,Άγιος Ανδρέας,5767,2817,2950,Λευκωσίας,False
6,100002,Τρυπιώτης,2158,983,1175,Λευκωσίας,False
7,100003,Νεμπέτ Χανέ,189,86,103,Λευκωσίας,False
8,100004,Ταμπάκ Χανέ,299,117,182,Λευκωσίας,False


## Remove suburbs from locations

This is done for several reasons:
- Have a consistent data structure: all locations are settlements, not suburbs
- Avoid double population counts

This behaviour can be switched off by setting `remove_suburbs=False`

In [11]:
remove_suburbs = True

# Remove suburbs
# The suburbs are the towns that have a six-digit geo code
if remove_suburbs:
    census_df = census_df[census_df['geo_code'].apply(lambda x: len(str(x)) < 6)].reset_index(drop=True)

In [12]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [5]:
# Set version
version = 1

In [13]:
# Save a checkpoint of the dataframe to a csv file
census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)

In [6]:
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

# 2) Google Maps API (Geocoding API)

Source: [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/overview?hl=en-419)

## Notes:
- The API key is stored in a file called `api_key.txt`. This file is not included in the repository. To reproduce the results here, you need to create your own API key and store it in a file called `api_key.txt` in the folder `private_utils`.
- Requests are limited to 2500 per day. 
- Retrieving data from the API is not free. Be aware of the costs when rebuilding the dataset.

In [6]:
# Read API key
with open('private_utils/api_key.txt', 'r') as f:
    api_key = f.read()

## Auxiliary functions

In [16]:
# Extract coordinates for towns in Cyprus from Geocoding API
def extract_coordinates(
        town: str,
        district: str = None,
        api_key: str = api_key, 
        boundaries: list = [34.51, 32.17, 35.73, 34.61]) -> tuple:
    """
    Extract coordinates for a town in Cyprus from Google Maps Geocoding API.

    The boundaries argument 

    Parameters
    ----------
    town : str
        The town name.
    district : str
        The district name.
    api_key : str
        The Google Maps API key.
    boundaries : list
        The boundaries of the search area. It is a list of four floats that represent the
        boundaries of the search area. The order of the floats is as follows:
        [southwest_lat, southwest_lon, northeast_lat, northeast_lon].
        By default, the boundaries are set to the approximate boundaries of Cyprus island.

    Returns
    -------
    lat : float
        The latitude of the town.
    lon : float
        The longitude of the town.
    gm_id : str
        The Google Maps ID of the town.
    """
        
    # Set search term
    query = town

    # Add district to search term
    if district is not None:
        query += ' ' + district

    # Set Geocoding API URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&key=' + api_key

    # Add boundaries to search
    if boundaries:
        url += '&bounds=' + str(boundaries[0]) + ',' + str(boundaries[1]) + '|' + str(boundaries[2]) + ',' + str(boundaries[3])

    # Extract coordinates
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract coordinates and Google Maps ID
    lat = data['results'][0]['geometry']['location']['lat']
    lon = data['results'][0]['geometry']['location']['lng']
    gm_id = data['results'][0]['place_id']

    return lat, lon, gm_id

## Data retrieval

In [17]:
census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True


In [7]:
# Generate coordinates if requested
generate_coordinates = False

if generate_coordinates:
    # Extract coordinates for towns in Cyprus from Geocoding API
    # Takes ~ 1 minute
    # Latitude, longitude, Google Maps ID
    census_df['lat'], census_df['lon'], census_df['gm_id'] = zip(*census_df.apply(lambda x: extract_coordinates(x['town'], distrinct = x['district']), axis=1))

    # Save coordinates with retrieval date
    census_coordinates = census_df[['town', 'district', 'lat', 'lon', 'gm_id']]
    census_coordinates.to_csv(f'sources/Geocoding API/geocoding_coordinates_ROC_{date.today().strftime("%Y-%m-%d")}.csv', index=False)

else:
    # Select the latest coordinates file
    list_of_files = glob.glob('sources/Geocoding API/geocoding_coordinates_ROC_*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Load coordinates from file
    census_coordinates = pd.read_csv(latest_file)

# Add coordinates to census dataframe
# census_df = census_df.merge(census_coordinates, on=['town', 'district'], how='left')
census_df[['lat', 'lon', 'gm_id']] = census_coordinates[['lat', 'lon', 'gm_id']]

census_df.head()

,geo_code,town,population,male_population,female_population,district,is_dimos,lat,lon,gm_id
0,1000,Λευκωσία,55014,26520,28494,Λευκωσίας,True,35.185566,33.382276,ChIJVU1JymcX3hQRpcARA5ykXls
1,1010,Άγιος Δομέτιος,12456,5861,6595,Λευκωσίας,True,35.172787,33.329092,ChIJqzUpSaEQ3hQRgmUX_emhREA
2,1011,Έγκωμη,18010,8547,9463,Λευκωσίας,True,35.153823,33.316954,ChIJtcdnJb8Q3hQR80ccwpjkIPk
3,1012,Στροβόλος,67904,32248,35656,Λευκωσίας,True,35.143663,33.343791,ChIJ90x2dika3hQRq7-H2HRHAJo
4,1013,Αγλαντζιά,20783,9803,10980,Λευκωσίας,True,35.149803,33.394086,ChIJb0vkuNMZ3hQR_4oSWBdFjX0


## Plot and inspect results

In [8]:
# Show the retrieval results to check for errors
# Plot all towns in Cyprus
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        tooltip=f"{census_df['town'][i]}",
        parse_html=False).add_to(map)
    
map

## Results notes

1) The coordinates for the town of Πόλη Χρυσοχούς (Póli Chrysochoús) are not correct. This is due to conflicting names with other natural places and locations and to the fact that the algorithm only takes the first result. Since it is the only instance of such a problem, a manual correction is applied.

In [21]:
# Prepare manual corrections 
coordinates_manual_mapping = {
    'Πόλις Χρυσοχούς' : [35.0339441, 32.4253751, 'ChIJG4zmlxR05xQR7lc0vj1h-YQ']
}

# Apply manual corrections
for town in coordinates_manual_mapping.keys():
    census_df.loc[census_df['town'] == town, ['lat', 'lon', 'gm_id']] = coordinates_manual_mapping[town]

## Inspect results after correction

In [9]:
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df)):
    folium.CircleMarker(
        location=[census_df['lat'][i], census_df['lon'][i]],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        # Add text with the town name when hovering over the marker
        tooltip=f"{census_df['town'][i]}",
        parse_html=False).add_to(map)
    
map

## Save checkpoint

In [23]:
# Save a checkpoint of the dataframe to a csv file
version = 2

census_df.to_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv', index=False)
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

In [24]:
# Load the checkpoint
census_df = pd.read_csv(f'checkpoints/CyprusDB_cp_v{str(version)}.csv')

# 3) Add census data for Northern Cyprus

Source: [2011 Census of Northern Cyprus](https://www.ktoeos.org/wp-content/uploads/2013/08/nufus_ikinci_.pdf)

The data is extracted from the above PDF. Target table is table 5, which contains population information for each town for years 2006 and 2011. Pages 25 to 33 contain the data. 

## Read pages

In [235]:
# Read each individual page from the PDF file
# The tables are in pages 25 to 33
pages = tabula.read_pdf('sources/TRNC Census 2011/nufus_ikinci_.pdf', pages='25-33')

# Delete the three first rows from every page
# They contain general information
for page in pages:
    page.drop(page.index[:3], inplace=True)

## Clean pages

In [236]:
# Concatenate all pages into a single dataframe
census_df_trnc = pd.concat(pages, ignore_index=True)

# Rename columns
# 'mixed_data' contains several columns together due to OCR errors 
column_names = ['town', 'population_2006', 'mixed_data', 'male_population', 'female_population']
census_df_trnc.columns = column_names

# Separete the columns in 'mixed_data'
new_columns = ['male_population_2006', 'female_population_2006', 'population']
census_df_trnc[new_columns] = census_df_trnc['mixed_data'].str.split(' ', expand=True)

# Reorder columns
order = ['town', 
         'population', 'male_population', 'female_population', 
         'population_2006', 'male_population_2006', 'female_population_2006']
census_df_trnc = census_df_trnc[order]

# Remove rows where the town is not specified
# Those rows are headers
census_df_trnc = census_df_trnc[census_df_trnc['town'].notna()]

# Convert selected columns to numeric
numeric_columns = [
    'population', 'male_population', 'female_population', 
    'population_2006', 'male_population_2006', 'female_population_2006']
for column in numeric_columns:
    # Remove ',' in numbers
    census_df_trnc[column] = census_df_trnc[column].str.replace(',', '').astype(int)

# Show all rows
census_df_trnc.head()

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006
1,Lefkoşa İlçe Toplamı,94824,49838,44986,84776,46187,38589
2,Lefkoşa Merkez Bucak Toplamı,82929,43628,39301,72479,39337,33142
3,Lefkoşa Belediye Toplamı,61378,32260,29118,56146,30583,25563
4,ABDİ ÇAVUŞ,568,315,253,975,591,384
5,AKKAVUK,793,458,335,898,498,400


## Extract districts, subdistricts and municipalities

In [237]:
# Create a columns named 'district'
# If 'İlçe' ('district') is in the town name, extract it
census_df_trnc['district'] = census_df_trnc['town'].apply(lambda x: x.split(' İlçe ')[0] if ' İlçe' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['district'] = census_df_trnc['district'].ffill()

In [238]:
# Create a column for subdistricts
# If 'Bucak' ('subdistrict') is in the town name, extract it
census_df_trnc['subdistrict'] = census_df_trnc['town'].apply(lambda x: x.split(' ')[0] if ' Bucak' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['subdistrict'] = census_df_trnc['subdistrict'].ffill() 

In [239]:
# Create a column for municipalities
# If 'Belediye' ('municipality') is in the town name, extract it
census_df_trnc['municipality'] = census_df_trnc['town'].apply(lambda x: x.split(' ')[0] if ' Belediye' in x else np.nan)

# Apply a forward fill to fill the missing values
census_df_trnc['municipality'] = census_df_trnc['municipality'].ffill()

## Final cleaning

In [240]:
# Remove rows including 'Toplamı' ('Total')
census_df_trnc = census_df_trnc.loc[~census_df_trnc['town'].str.contains('Toplamı')]

Certain towns requiere name corrections in order to be processed by the Geocoding API in future steps. Otherwise, the coordinates are not retrieved.

* **KAPALI MARAŞ** ("Closed Maraş"): MARAŞ
* **MALATYA - İNCESU**: İNCESU
* **KARAMAN (YUKARI KARMİ)** ("Upper Karmi") : KARAMAN 

In [242]:
# Set towns to correct
town_corrections = {
    # Raw census name : Corrected name
    'KAPALI MARAŞ' : 'MARAŞ',
    'MALATYA - İNCESU' : 'İNCESU',
    'KARAMAN (YUKARI KARMİ)' : 'KARAMAN'
}

# Apply corrections
for old_name, new_name in town_corrections.items():
    census_df_trnc.loc[census_df_trnc['town'] == old_name, 'town'] = new_name

## Save checkpoint

In a secondary folder prior to merger with the main dataset

In [243]:
# Set version
version = 1

In [244]:
# Save a checkpoint of the dataframe to a csv file
census_df_trnc.to_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv', index=False)

In [245]:
# Load the checkpoint
census_df_trnc = pd.read_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv')

# 5) TRNC post codes: quarter merging

To obtain accuarate Google Maps IDs in the next section and homogenize the criteria with ROC data, quarters are merged into the city they belong to.

Since there is no guarantee that the quarters will be named the same way in the two datasets, the quarters are extracted alongside with the possible city they are adscribed, extracted from the `bucak` ("subdistrict") column. Only rows containing 'MAH' ("quarter") are preserved. Finally, a manual check is performed to ensure that all quarters are correctly assigned to the city they belong to.

**Note**: the original file uses an `ı` in the name, which is not read by Python. Thus, a new file with an `i` is created.

## Read pages

In [246]:
# Read each individual page from the PDF file
postal_codes_pages = tabula.read_pdf('sources\Postal codes TRNC/KKTC Posta Kodlari.pdf', pages = 'all')

# Concatenate all pages into a single dataframe
raw_postal_codes_df = pd.concat(postal_codes_pages, ignore_index=True)

## Clean pages

In [247]:
# Create a copy of the dataframe
postal_codes_df = raw_postal_codes_df.copy()

# Drop columns contaning 'İLÇESİ'
postal_codes_df.drop(columns=postal_codes_df.columns[postal_codes_df.columns.str.contains('İLÇESİ')], inplace=True)

# Drop the first column
postal_codes_df.drop(columns=postal_codes_df.columns[0], inplace=True)

# Rename columns
column_renames = {
    'Unnamed: 0': 'district',
    'Unnamed: 1': 'subdistrict',
    'Unnamed: 2': 'entity',
    'Unnamed: 3': 'postal_code'
}
postal_codes_df.rename(columns=column_renames, inplace=True)

# Remove rows where the district is not specified
# Those rows are headers
postal_codes_df = postal_codes_df[postal_codes_df['district'].notna()]

# Remove rows where the postal code is 'PK'
# Those rows are headers
postal_codes_df = postal_codes_df[postal_codes_df['postal_code'] != 'PK']

# Replace GAZİ.MAĞUSA with GAZİMAĞUSA
postal_codes_df['subdistrict'] = postal_codes_df['subdistrict'].str.replace('GAZİ.MAĞUSA', 'GAZİMAĞUSA', regex = False)

# Replace G.MAĞUSA with GAZİMAĞUSA in the district column
postal_codes_df['district'] = postal_codes_df['district'].str.replace('G.MAĞUSA', 'GAZİMAĞUSA', regex = False)

## Homogenize district names

Altough visually equal, the district names are not equal in the two datasets in terms of ASCII characters. To simplify the mergind process, a manual mapping between the two datasets is required.

In [248]:
# Show differences
# display([ord(c) for c in 'Gazimağusa'], 
#         [ord(c) for c in 'GAZİMAĞUSA'.lower()])

# Create a dictionary to map the district names
district_ascii_mapppings = {
    'LEFKOŞA' : 'Lefkoşa',
    'GİRNE' : 'Girne',
    'GAZİMAĞUSA' : 'Gazimağusa',
    'GÜZELYURT' : 'Güzelyurt',
    'İSKELE' : 'İskele'
}

# Map the district names
postal_codes_df['district'] = postal_codes_df['district'].map(district_ascii_mapppings)

In [249]:
# Show table
postal_codes_df.head()

,district,subdistrict,entity,postal_code
2,Lefkoşa,LEFKOŞA,ABDİ ÇAVUŞ MAH,99010
3,Lefkoşa,LEFKOŞA,AKKAVUK MAH,99010
4,Lefkoşa,LEFKOŞA,ARABAHMET MAH,99010
5,Lefkoşa,LEFKOŞA,AYDEMET MAH,99010
6,Lefkoşa,LEFKOŞA,AYYILDIZ MAH,99010


## Quarter merging

It is assumed that if the `entity` column contains 'MAH' or 'MAHALLESİ', it is a quarter of the town that appears in the `bucak` column.

Exceptions detected through manual inspection:
- `İPLİK PAZARI/K.EFENDİ` is a quarter of `Lefkoşa`
- `DENİZLİ` is considered a separate town from GEMİKONAĞI in Google Maps and as its own ID.


Further manual inspection might be necessary to detect more exceptions.

In [250]:
# Manual corrections prior to merging

# Add ' MAH' at the end of the row contaning 'İPLİK PAZARI/K.EFENDİ'
postal_codes_df.loc[postal_codes_df['entity'] == 'İPLİK PAZARI/K.EFENDİ', 'entity'] = 'İPLİK PAZARI/K.EFENDİ MAH'

# Replace 'MAH' with 'KÖYÜ' in the row containing 'DENİZLİ MAH' to avoif being merged with 'GEMİKONAĞI MAH'
postal_codes_df.loc[postal_codes_df['entity'] == 'DENİZLİ MAH', 'entity'] = 'DENİZLİ KÖYÜ'

# Create a backup of the dataframe
postal_codes_df_backup = postal_codes_df.copy()

In [251]:
# Create a copy of the base dataframe
postal_codes_df = postal_codes_df_backup.copy()

# Drop all rows where the entity does not contain 'MAH'
postal_codes_df = postal_codes_df[postal_codes_df['entity'].str.contains('MAH')]

# Create a new columns called 'quarter_name' and populate it with the entity name removing the last word
postal_codes_df['quarter_name'] = postal_codes_df['entity'].apply(lambda x: ' '.join(x.split(' ')[:-1]))

postal_codes_df

,district,subdistrict,entity,postal_code,quarter_name
2,Lefkoşa,LEFKOŞA,ABDİ ÇAVUŞ MAH,99010,ABDİ ÇAVUŞ
3,Lefkoşa,LEFKOŞA,AKKAVUK MAH,99010,AKKAVUK
4,Lefkoşa,LEFKOŞA,ARABAHMET MAH,99010,ARABAHMET
5,Lefkoşa,LEFKOŞA,AYDEMET MAH,99010,AYDEMET
6,Lefkoşa,LEFKOŞA,AYYILDIZ MAH,99010,AYYILDIZ
7,Lefkoşa,LEFKOŞA,ÇAĞLAYAN MAH,99010,ÇAĞLAYAN
8,Lefkoşa,LEFKOŞA,HAYDARPAŞA MAH,99010,HAYDARPAŞA
9,Lefkoşa,LEFKOŞA,GÖÇMENKÖY MAH,99010,GÖÇMENKÖY
10,Lefkoşa,LEFKOŞA,İBRAHİM PAŞA MAH,99010,İBRAHİM PAŞA
11,Lefkoşa,LEFKOŞA,İPLİK PAZARI/K.EFENDİ MAH,99010,İPLİK PAZARI/K.EFENDİ


In [252]:
# Create a new base dataframe to ensure repeatability
clean_postal_codes_df = postal_codes_df.copy()

# Create a new column indicating whether the quarter is included in the general census
clean_postal_codes_df['included_in_census'] = clean_postal_codes_df['quarter_name'].isin(census_df_trnc['town'])

# Set the index to the quarter name
clean_postal_codes_df.set_index('quarter_name', inplace=True)

clean_postal_codes_df

,district,subdistrict,entity,postal_code,included_in_census
quarter_name,,,,,
ABDİ ÇAVUŞ,Lefkoşa,LEFKOŞA,ABDİ ÇAVUŞ MAH,99010,True
AKKAVUK,Lefkoşa,LEFKOŞA,AKKAVUK MAH,99010,True
ARABAHMET,Lefkoşa,LEFKOŞA,ARABAHMET MAH,99010,True
AYDEMET,Lefkoşa,LEFKOŞA,AYDEMET MAH,99010,True
AYYILDIZ,Lefkoşa,LEFKOŞA,AYYILDIZ MAH,99010,True
ÇAĞLAYAN,Lefkoşa,LEFKOŞA,ÇAĞLAYAN MAH,99010,False
HAYDARPAŞA,Lefkoşa,LEFKOŞA,HAYDARPAŞA MAH,99010,True
GÖÇMENKÖY,Lefkoşa,LEFKOŞA,GÖÇMENKÖY MAH,99010,True
İBRAHİM PAŞA,Lefkoşa,LEFKOŞA,İBRAHİM PAŞA MAH,99010,False


## Modify the general TRNC census dataframe adding quarter information from the postal codes dataframe

In [253]:
# Add a new column in the general TRNC census dataframe indicating the main city-subdistrict (bucak) to which the quarter belongs
def get_main_city(town: str,
                  district: str,
                  quarter_df = clean_postal_codes_df) -> str:
    
    # Filter by district first to avoid duplicates
    # The district must be uppercase to match formats
    district_df = quarter_df.loc[quarter_df['district'] == district]

    # Get the main city to which it belongs, which appeares in the 'subdistrict' column
    if town in district_df.index:
        main_city = district_df.loc[town, 'subdistrict']
        
    else:
        main_city = np.nan

    return main_city

census_df_trnc['main_city'] = census_df_trnc.apply(lambda x: get_main_city(x['town'], x['district']), axis=1)

In [254]:
# Show results
census_df_trnc.head()

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,main_city
0,ABDİ ÇAVUŞ,568,315,253,975,591,384,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
1,AKKAVUK,793,458,335,898,498,400,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
2,ARABAHMET,561,297,264,761,425,336,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
3,AYDEMET,2314,1147,1167,1550,765,785,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
4,AYYILDIZ,489,271,218,559,316,243,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA


### Quarter manual correction

Not all quarters are included in the census or have not been detected. Thus, a manual correction is required. 

The correspondences are established by manually comparing the two datasets: all the quarters in `clean_postal_codes_df` with a value of `False` for the `included_in_census` column are manually checked against the town names in the `census_df_trnc` census dataset to detect naming inconsistencies and assign a main city to them.

In [255]:
# Create a dictionary with the quarters to add to each main city
quarter_corrections = {
    # Main city : quarters to add (namings as shown in the census_df_trnc dataframe)
    # Nicosia
    'LEFKOŞA' : ['ÇAĞLAYAN',
                 'İBRAHİMPAŞA',
                 'İPLİKPAZARI',
                 'KÖŞKLÜÇİFTLİK',
                 'MAHMUTPAŞA'],
    # Kioneli (Gönyeli)
    'GÖNYELİ' : ['YENİKENT'],
    # Kyrenia (Girne)
    # This instance may be confused with 'ZEYTİNLİK KÖY'. 
    # Differences are to be determined and might be reviewed in the future.
    'GİRNE' : ['ZEYTİNLİK KESİM'], 
    # Alsancak
    'ALSANCAK' : ['YAYLA'],
    # Lapithos (Lapta)
    'LAPTA' : ['SAKARYA',
               'YAVUZ'],
    # Famagusta (Gazimağusa)
    'GAZİMAĞUSA' : ['CANBOLAT',
                    'SURİÇİ'],
    # Tríkomo (İskele)
    'İSKELE' : ['CEVİZLİ'],
    # Rizokarpaso (Dipkarpaz)
    'DİPKARPAZ' : ['POLAT PAŞA']                
}

In [256]:
# Add the quarters to the main cities
for main_city, quarters in quarter_corrections.items():
    for quarter in quarters:
        census_df_trnc.loc[census_df_trnc['town'] == quarter, 'main_city'] = main_city

# Some quarter names are shared between different main cities and towns. 
# However, the above code assigns all quarters with the same name to the same main city.
# To account for this, the following manual corrections are needed:
quarter_extra_corrections = [
    # Settlement, Municipality, Main city value
    ('YAYLA', 'Güzelyurt', np.nan),
    ('ÇAĞLAYAN', 'Alsancak', 'ALSANCAK'),
    ('SAKARYA', 'Gazimağusa', 'GAZİMAĞUSA')
]

for settlement, municipality, main_city_value in quarter_extra_corrections:
    census_df_trnc.loc[(census_df_trnc['town'] == settlement) & (census_df_trnc['municipality'] == municipality), 'main_city'] = main_city_value    

census_df_trnc

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,main_city
0,ABDİ ÇAVUŞ,568,315,253,975,591,384,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
1,AKKAVUK,793,458,335,898,498,400,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
2,ARABAHMET,561,297,264,761,425,336,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
3,AYDEMET,2314,1147,1167,1550,765,785,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
4,AYYILDIZ,489,271,218,559,316,243,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
5,ÇAĞLAYAN,1307,667,640,1413,744,669,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
6,GÖÇMENKÖY,3003,1551,1452,2946,1526,1420,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA
7,HAMİTKÖY,5338,2773,2565,2898,1567,1331,Lefkoşa,Lefkoşa,Lefkoşa,NaN
8,HASPOLAT,4204,2385,1819,3380,2168,1212,Lefkoşa,Lefkoşa,Lefkoşa,NaN
9,HAYDARPAŞA,155,80,75,320,186,134,Lefkoşa,Lefkoşa,Lefkoşa,LEFKOŞA


Certain locations are divided in "upper" ("Yukarı") and "lower" ("Aşağı") parts. To help with coordinate retrieval in future steps, those instances will be assigned a main city, so that they are merged into a single entity recognized by the Geocoding API.

Divided locations:
- Girne
- Dikmen

In [257]:
divided_locations = ['GİRNE', 'DİKMEN']
location_prefixes = ['YUKARI', 'AŞAĞI']

for location in divided_locations:
    for prefix in location_prefixes:
        census_df_trnc.loc[census_df_trnc['town'] == f'{prefix} {location}', 'main_city'] = location

Additionally, the city of Trikomo (İskele) has an entity in the census, but it has no `main_city` value. To improve the quarter merging process, it is set to the city name.

In [258]:
# Add İSKELE as a main city to its row in the census dataframe
census_df_trnc.loc[census_df_trnc['town'] == 'İSKELE', 'main_city'] = 'İSKELE'

## Merge all quarters to their main cities

To do so, the values for all the rows that share a `main_city` value in `census_df_trnc` are summed up, resulting in a new row for the city is created.

There are two possibilities for these values:
1) There is no instance in the `town` column with the name of the city in the census. In this case, the row is added to the dataframe and all the rows are summed up.
2) There is an instance with that name in census dataset, further divided in two cases:
   - There is only one instance that has that name as its `main_city` that name. In this case, the process is skipped.
   - There is more than one instance that has that name as its `main_city`. In this case, the rows are summed up and the result is added to the dataframe.

All these cases are handled by the `merge_quarter_data` function.

In [259]:
# Get unique values for the main cities without NaN
main_cities = [city for city in census_df_trnc['main_city'].unique() if city is not np.nan]
print(f'Main cities: {main_cities}')

# Create an auxiliary dataframe to store the results
aux_census_df_trnc = census_df_trnc.copy()

Main cities: ['LEFKOŞA', 'GÖNYELİ', 'DEĞİRMENLİK', 'GAZİMAĞUSA', 'TATLISU', 'GİRNE', 'ALSANCAK', 'DİKMEN', 'LAPTA', 'GÜZELYURT', 'GEMİKONAĞI', 'İSKELE', 'DİPKARPAZ']


### Merge main cities that have no instance in the census

In [260]:
def merge_quarter_data(
        cities: list, 
        census_df: pd.DataFrame = aux_census_df_trnc) -> pd.DataFrame:
    """
    Computes the total population for each main city and adds administrative information 
    based on a list of main cities. It also deletes the information for the quarters that
    belong to the main cities.

    Parameters
    ----------
    cities : list
        List of main cities to be processed.
    census_df : pd.DataFrame
        Dataframe containing the census data. By default,
        the auxiliary dataframe is used to avoid overwriting
        the original data.

    Returns
    -------
    census_df: pd.DataFrame
        A modified version of the original dataframe with the
        population data for each main city. 
    """

    for city in cities:
        # Extract city quarters
        city_filter = census_df[census_df['main_city'] == city]

        # Drop all rows for which `main_city` is `city`
        census_df = census_df[census_df['main_city'] != city]

        # Store administrative information
        city_admin_info = city_filter[['district', 'subdistrict', 'municipality', 'main_city']].copy()
        # Use this method to confirm that data belong to the same district and municipality
        # If two rows are returned, the data is not consistent
        city_admin_info.drop_duplicates(inplace=True) 

        # Sum population rows
        city_population = (city_filter
                        .groupby('main_city')
                        .sum()
                        .reset_index())

        # Add administrative information
        city_info = pd.merge(city_population, city_admin_info, on='main_city')

        # Extract town name from 'main_city' column
        city_info['town'] = city_info['main_city']

        # Reorder columns
        order = [
            'town', 
            'population', 'male_population', 'female_population', 
            'population_2006', 'male_population_2006', 'female_population_2006',
            'district', 'subdistrict', 'municipality', 'main_city'
            ]
        city_info = city_info[order]

        # Add city to the census
        census_df = census_df.append(city_info)

    # Drop 'main_city' column
    census_df.drop(columns='main_city', inplace=True)

    return census_df

In [261]:
# Apply quarter merging function
aux_census_df_trnc = merge_quarter_data(main_cities, aux_census_df_trnc)

# Visual check previous to overwriting the original dataframe
aux_census_df_trnc.tail(12)

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality
0,GÖNYELİ,5374,2725,2649,3478,1808,1670,Lefkoşa,Lefkoşa,Gönyeli
0,DEĞİRMENLİK,3284,1778,1506,3128,1685,1443,Lefkoşa,Değirmenlik,Değimenlik
0,GAZİMAĞUSA,37642,20301,17341,33001,17897,15104,Gazimağusa,Gazimağusa,Gazimağusa
0,TATLISU,1120,579,541,903,461,442,Gazimağusa,Geçitkale,Tatlısu
0,GİRNE,21319,11403,9916,18744,10594,8150,Girne,Girne,Girne
0,ALSANCAK,5595,2948,2647,4638,2577,2061,Girne,Girne,Alsancak
0,DİKMEN,3969,2131,1838,2605,1464,1141,Girne,Girne,Dikmen
0,LAPTA,5748,2959,2789,5658,3151,2507,Girne,Girne,Lapta
0,GÜZELYURT,7251,3619,3632,7627,3885,3742,Güzelyurt,Güzelyurt,Güzelyurt
0,GEMİKONAĞI,2075,1318,757,1498,854,644,Güzelyurt,Lefke,Lefke


In [262]:
# Overwrite the original dataframe
census_df_trnc = (aux_census_df_trnc
                  .sort_index()
                  .reset_index(drop=True)
                  .copy())

census_df_trnc.head()

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality
0,DİPKARPAZ,2016,1007,1009,1935,968,967,İskele,Yeni,Dipkarpaz
1,GEMİKONAĞI,2075,1318,757,1498,854,644,Güzelyurt,Lefke,Lefke
2,GÜZELYURT,7251,3619,3632,7627,3885,3742,Güzelyurt,Güzelyurt,Güzelyurt
3,LAPTA,5748,2959,2789,5658,3151,2507,Girne,Girne,Lapta
4,DİKMEN,3969,2131,1838,2605,1464,1141,Girne,Girne,Dikmen


## Save checkpoint

In [263]:
# Save a checkpoint of the dataframe to a csv file
version = 2

census_df_trnc.to_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv', index=False)

In [264]:
# Load the checkpoint
census_df_trnc = pd.read_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv')

## Data cleaning prior to adding new sources

This mainly involves lowercasing strings, since TRNC data sources use capital letters for town naming.

### Lowercase town names

We can not apply a simple lowercase function, Since 'ı' is merged with 'i'. Consequently, we need to first replace 'I' with 'ı' and then lowercase the whole string.


In [265]:
def lowercase_turkish_town_name(town: str) -> str:
    """Lowercases the Turkish town names in the dataframe.

    Parameters
    ----------
    town : str
        The town name to be lowercased.

    Returns
    -------
    str
        The lowercased town name.

    """

    # Split the town name into words
    words = town.split(' ')
    
    processed_words = []
    for word in words:
        # Separate word parts
        initial = word[0]
        body = word[1:]

        # Replace 'I' with 'ı' in the body, then lowercase it
        body = body.replace('I', 'ı').lower()

        # Join the word parts back together
        word = initial + body

        processed_words.append(word)
        

    # Join the words back into a string
    town = ' '.join(processed_words)

    return town

In [266]:
# Apply lowercasing function
census_df_trnc['town'] = census_df_trnc['town'].apply(lowercase_turkish_town_name)

## Save checkpoint

In [267]:
# Save a checkpoint of the dataframe to a csv file
version = 3

census_df_trnc.to_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv', index=False)

In [268]:
# Load the checkpoint
census_df_trnc = pd.read_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv')

# 6) Google Maps API for northern Cyprus

### Important notes

Data for settlements in northern Cyprus is heavily unreliable: the coordinates offered point to the center of the administrative unit and not to the settlement itself. Altough the coordinates can be obtained through OpenStreetMap API, the Google Maps API is still needed to get consistent Greek and Turkish namings for towns. As such, coordinates will be generated and will be overwritten by OpenStreetMap data later on.

In [269]:
# Extract coordinates for towns in Cyprus from Geocoding API
def extract_coordinates(
        town: str,
        district: str = None,
        language: str = None,
        api_key: str = api_key, 
        boundaries: list = [34.51, 32.17, 35.73, 34.61]) -> tuple:
    """
    Extract coordinates for a town in Cyprus from Google Maps Geocoding API.

    The boundaries argument 

    Parameters
    ----------
    town : str
        The town name.
    district : str
        The district name.
    api_key : str
        The Google Maps API key.
    boundaries : list
        The boundaries of the search area. It is a list of four floats that represent the
        boundaries of the search area. The order of the floats is as follows:
        [southwest_lat, southwest_lon, northeast_lat, northeast_lon].
        By default, the boundaries are set to the approximate boundaries of Cyprus island.

    Returns
    -------
    lat : float
        The latitude of the town.
    lon : float
        The longitude of the town.
    gm_id : str
        The Google Maps ID of the town.
    """
        
    # Set search term
    query = town

    # Add district to search term
    if district is not None:
        query += ' ' + district
        print(query)

    # Set Geocoding API URL
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + '&key=' + api_key

    # Add boundaries to search
    if boundaries:
        url += '&bounds=' + str(boundaries[0]) + ',' + str(boundaries[1]) + '|' + str(boundaries[2]) + ',' + str(boundaries[3])
    
    # Add language
    if language is not None:
        query += ' ' + language

    ### Request execution ###

    # Extract coordinates
    response = requests.get(url)
    data = json.loads(response.text)
    
    # Extract coordinates and Google Maps ID
    try:
        
        # If there are no results with 'locality', return NaNs
        if len(data['results']) == 0:
            print('Unable to extract coordinates for ' + town + ': no results found.')
            return np.nan, np.nan, np.nan

        # Get the first results that contains 'locality' in the types list and
        # is within the boundaries
        valid_i = None
        for i, result in enumerate(data['results']):
            # Check that the result contains 'locality'
            if 'locality' in result['types']:
                # Check that the result is within the boundaries
                lat = data['results'][i]['geometry']['location']['lat']
                lon = data['results'][i]['geometry']['location']['lng']
                if lat < boundaries[0] or lat > boundaries[2] or lon < boundaries[1] or lon > boundaries[3]:
                    continue

                # If all checks are valid, set the index of the result
                valid_i = i
                break

            # If there are no results, return NaNs
            elif i == len(data['results']) - 1:
                print('Unable to extract coordinates for ' + town + ': no locality results found.')
                return np.nan, np.nan, np.nan
            
        # If there are no valid results, return NaNs
        if valid_i is None:
            print('Unable to extract coordinates for ' + town + ': no valid results found.')
            return np.nan, np.nan, np.nan

        # Extract coordinates and Google Maps ID
        lat = data['results'][valid_i]['geometry']['location']['lat']
        lon = data['results'][valid_i]['geometry']['location']['lng']
        gm_id = data['results'][valid_i]['place_id']

    except ValueError:
        print('Unable to extract coordinates for ' + town + '. General error')
        lat = np.nan
        lon = np.nan
        gm_id = np.nan

    return lat, lon, gm_id

In [270]:
# Generate coordinates if requested
generate_coordinates = True

if generate_coordinates:
    # Extract coordinates for towns in Cyprus from Geocoding API
    # Takes ~ 1 minute
    # Latitude, longitude, Google Maps ID
    census_df_trnc['lat'], census_df_trnc['lon'], census_df_trnc['gm_id'] = zip(*census_df_trnc.apply(lambda x: extract_coordinates(x['town'], 
                                                        district = 'Kıbrıs',
                                                        boundaries=[34.9, 32.5, 35.8, 34.6]), axis=1))

    # Save coordinates with retrieval date
    census_coordinates = census_df_trnc[['town', 'district', 'lat', 'lon', 'gm_id']]
    census_coordinates.to_csv(f'sources/Geocoding API/geocoding_coordinates_TRNC_{date.today().strftime("%Y-%m-%d")}.csv', index=False)

else:
    # Select the latest coordinates file
    list_of_files = glob.glob('sources/Geocoding API/geocoding_coordinates_TRNC_*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    
    # Load coordinates from file
    census_coordinates = pd.read_csv(latest_file)

# Add coordinates to census dataframe
# census_df = census_df.merge(census_coordinates, on=['town', 'district'], how='left')
census_df_trnc[['lat', 'lon', 'gm_id']] = census_df_trnc[['lat', 'lon', 'gm_id']]

census_df_trnc.head()

Di̇pkarpaz Kıbrıs
Gemi̇konağı Kıbrıs
Güzelyurt Kıbrıs
Lapta Kıbrıs
Di̇kmen Kıbrıs
Alsancak Kıbrıs
Gi̇rne Kıbrıs
Unable to extract coordinates for Gi̇rne: no locality results found.
Tatlısu Kıbrıs
Gazi̇mağusa Kıbrıs
Deği̇rmenli̇k Kıbrıs
Gönyeli̇ Kıbrıs
Lefkoşa Kıbrıs
İskele Kıbrıs
Hami̇tköy Kıbrıs
Haspolat Kıbrıs
Akıncılar Kıbrıs
Alayköy Kıbrıs
Türkeli̇ Kıbrıs
Yılmazköy Kıbrıs
Gönyeli̇ Kıbrıs
Kanlıköy Kıbrıs
Balıkesi̇r Kıbrıs
Beyköy Kıbrıs
Ci̇hangi̇r Kıbrıs
Çukurova Kıbrıs
Demi̇rhan Kıbrıs
Di̇lekkaya Kıbrıs
Düzova Kıbrıs
Erdemli̇ Kıbrıs
Gazi̇köy Kıbrıs
Gökhan Kıbrıs
Kalavaç Kıbrıs
Kırıkkale Kıbrıs
Kırklar Kıbrıs
Meri̇ç Kıbrıs
Mi̇nareli̇köy Kıbrıs
Taşocakları Kıbrıs
Unable to extract coordinates for Taşocakları: no locality results found.
Yeni̇ceköy Kıbrıs
Yi̇ği̇tler Kıbrıs
Maraş Kıbrıs
Unable to extract coordinates for Maraş: no locality results found.
Mutluyaka Kıbrıs
Tuzla Kıbrıs
Yeni̇ Boğazi̇çi̇ Kıbrıs
Akova Kıbrıs
Alani̇çi̇ Kıbrıs
Atlılar Kıbrıs
Mormenekşe Kıbrıs
Muratağa Kıbrıs
San

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,lat,lon,gm_id
0,Di̇pkarpaz,2016,1007,1009,1935,968,967,İskele,Yeni,Dipkarpaz,35.617682,34.408731,ChIJFdx3t2tn3xQRInHdNFtCOCg
1,Gemi̇konağı,2075,1318,757,1498,854,644,Güzelyurt,Lefke,Lefke,35.142762,32.811270,ChIJw1gDZOdY5xQRzgnyO-yf0b4
2,Güzelyurt,7251,3619,3632,7627,3885,3742,Güzelyurt,Güzelyurt,Güzelyurt,35.212317,32.977567,ChIJfz7P_1H53RQRmbROZQ0T7bg
3,Lapta,5748,2959,2789,5658,3151,2507,Girne,Girne,Lapta,35.336408,33.163266,ChIJSfTuTocM3hQRNbYv5mkqRUM
4,Di̇kmen,3969,2131,1838,2605,1464,1141,Girne,Girne,Dikmen,35.266918,33.327700,ChIJf0s8-cAT3hQRY6tDs7XxiJg


In [232]:
## Plot and inspect results
# Show the retrieval results to check for errors
# Plot all towns controlled by the TRNC
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df_trnc)):
    try:
        folium.CircleMarker(
            location=[census_df_trnc['lat'][i], census_df_trnc['lon'][i]],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False,
            # Add text when hovering over the marker
            tooltip=f"{census_df_trnc['town'][i]}",
            ).add_to(map)
    except ValueError:
        pass
    
map

## Error correction

After a visual inspection of the results, the following errors were found:
- Several towns were not found in the Geocoding API. Possible reasons include:
    - Several possible results for a unique location were returned.
    - No results were returned. Specifically, the register named `Taşocakları` does not exist and seems to refer to a quarry. Further research is required to determine the nature of this instance.
- The town of Makrásika (`İncirli`), near Famagusta, receives the coordinates for an homonymous quarter in Lefkosia.

### Correct missing coordinates

In [272]:
# Show towns with missing coordinates
census_df_trnc[census_df_trnc['lat'].isna()]

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,lat,lon,gm_id
6,Gi̇rne,21319,11403,9916,18744,10594,8150,Girne,Girne,Girne,NaN,NaN,NaN
36,Taşocakları,0,0,0,128,91,37,Lefkoşa,Değirmenlik,Değimenlik,NaN,NaN,NaN
39,Maraş,226,125,101,503,317,186,Gazimağusa,Gazimağusa,Gazimağusa,NaN,NaN,NaN
81,Küçükerenköy,339,168,171,257,187,70,Gazimağusa,Geçitkale,Tatlısu,NaN,NaN,NaN
84,Doğanköy,868,429,439,457,245,212,Girne,Girne,Girne,NaN,NaN,NaN
106,Esentepe,1754,920,834,1575,949,626,Girne,Girne,Esentepe,NaN,NaN,NaN
147,Yedi̇dalga,669,329,340,767,397,370,Güzelyurt,Lefke,Lefke,NaN,NaN,NaN
156,Boğaztepe,312,151,161,444,280,164,İskele,İskele,İskele,NaN,NaN,NaN
180,Kantara,21,19,2,3,2,1,İskele,Mehmetçik,Mehmetçik,NaN,NaN,NaN


In [285]:
# Prepare manual corrections 
coordinates_manual_mapping = {
    # Town : [lat, lon, gm_id]
    # Unable to disambiguate results
    'Gi̇rne' : [35.299194, 33.2363246, 'ChIJG4zmlxR05xQR7lc0vj1h-YQ'], 
    
    # Unable to retrieve coordinates automatically
    'Maraş' : [35.105872, 33.9554907, 'ChIJFbxuNW_I3xQRXL-W8cUUcHc'],
    'Küçükerenköy': [35.3630139, 33.6682723, 'ChIJASOqhDZO3hQRLBc-qDsiiEI'],
    'Yedi̇dalga' : [35.144474, 32.8056285, 'ChIJcb1P3tJY5xQRmfZXUDXRy0s'],
    'Boğaztepe' : [35.3169997, 33.9457337, 'ChIJpZeEvrmv3xQRkQn98z8idbo'],
    'Kantara' : [35.3871012, 33.8990248, 'ChIJjZyEuU2p3xQROD6uAAxEHCg'],
    
    # Difficulty accessing Geocoding API instance: unable to find using references to Cyprus
    'Doğanköy' : [35.3241561, 33.3331825, 'ChIJ2_cKfkwT3hQRPMWDanLRPCQ'],

    # There is no entity in Google Maps for Esentepe (Agios Amvrosios)
    # Approximate coordinates from Google Maps are used
    'Esentepe' : [35.3403306, 33.5817749, np.nan],

    # Has the same name as a quarter in Nicosia
    'İnci̇rli̇' : [35.0795529, 33.7685332, 'ChIJwaHU1-Iy3hQRbQRoPDGgLv4']}

# Apply manual corrections
for town in coordinates_manual_mapping.keys():
    census_df_trnc.loc[census_df_trnc['town'] == town, ['lat', 'lon', 'gm_id']] = coordinates_manual_mapping[town]

In [288]:
# Show all rows
columns = ['town', 'population', 'lat', 'lon', 'gm_id']

# census_df_trnc[columns]
census_df_trnc.head()

,town,population,male_population,female_population,population_2006,male_population_2006,female_population_2006,district,subdistrict,municipality,lat,lon,gm_id
0,Di̇pkarpaz,2016,1007,1009,1935,968,967,İskele,Yeni,Dipkarpaz,35.617682,34.408731,ChIJFdx3t2tn3xQRInHdNFtCOCg
1,Gemi̇konağı,2075,1318,757,1498,854,644,Güzelyurt,Lefke,Lefke,35.142762,32.811270,ChIJw1gDZOdY5xQRzgnyO-yf0b4
2,Güzelyurt,7251,3619,3632,7627,3885,3742,Güzelyurt,Güzelyurt,Güzelyurt,35.212317,32.977567,ChIJfz7P_1H53RQRmbROZQ0T7bg
3,Lapta,5748,2959,2789,5658,3151,2507,Girne,Girne,Lapta,35.336408,33.163266,ChIJSfTuTocM3hQRNbYv5mkqRUM
4,Di̇kmen,3969,2131,1838,2605,1464,1141,Girne,Girne,Dikmen,35.266918,33.327700,ChIJf0s8-cAT3hQRY6tDs7XxiJg


## Results inspection after manual corrections

In [289]:
## Plot and inspect results
map = folium.Map(location=[35.1264, 33.4299], zoom_start=9)

for i in range(len(census_df_trnc)):
    try:
        folium.CircleMarker(
            location=[census_df_trnc['lat'][i], census_df_trnc['lon'][i]],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False,
            # Add text when hovering over the marker
            tooltip=f"{census_df_trnc['town'][i]}",
            ).add_to(map)
    except ValueError:
        pass
    
map

## Save checkpoint

In [ ]:
# Save a checkpoint of the dataframe to a csv file
version = 4

census_df_trnc.to_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv', index=False)

In [ ]:
# Load the checkpoint
census_df_trnc = pd.read_csv(f'checkpoints/secondary_checkpoints/TRNC_census_cp_v{str(version)}.csv')